## Аналитика
**Рассчитайте следующие показатели:**

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('Аналитика_1_СемашкоМА.xlsx', sheet_name = 'Маркетинговые данные')
df.head()

,Client ID,Domain,City,Region,Browser,Device Category,Date,Pageviews,Campaign,Keyword,...,Bounce Rate,Sessions,Goal Completion Location,Goal Completions,Goal Conversion Rate,Goal Value,Конверсия,Source conv.,Марка,Модель
0,GA1.2.724365598.1580545027,Mersedes-krasnodar.ru,Novy Urengoy,Yamalo-Nenets Autonomous Okrug,YaBrowser,desktop,2020-02-14,12,(not set),(not set),...,0.0,1,(entrance),1,1.0,9268,1,yandex.ru,Mercedes,NaN
1,GA1.2.1635604890.1580453987,Mersedes-krasnodar.ru,Salekhard,Yamalo-Nenets Autonomous Okrug,Safari,mobile,2020-02-14,2,(not set),(not provided),...,0.0,1,/models/cla/options/,0,0.0,0,0,google,Mercedes,cla
2,GA1.2.105232457.1578760776,Mersedes-krasnodar.ru,Novy Urengoy,Yamalo-Nenets Autonomous Okrug,Chrome,mobile,2020-02-14,2,(not set),(not provided),...,0.0,1,/models/gls/options/,1,1.0,32988,1,google,Mercedes,gls
3,GA1.2.774208163.1576953632,BMW-keyauto-krd.ru,Voronezh,Voronezh Oblast,Samsung Internet,mobile,2020-01-28,2,krasnodar-kliuchavto-network-halva-2019,(not set),...,0.0,1,/offers/buy/halva/,0,0.0,0,0,google_go_search,BMW,NaN
4,GA1.2.916661414.1580492871,Mersedes-krasnodar.ru,Voronezh,Voronezh Oblast,Chrome,mobile,2020-01-27,4,(not set),(not set),...,0.0,1,/buy/calc/?model=cla,1,1.0,27647,1,(direct),Mercedes,cla


In [3]:
# Из каких регионов больше всего заявок
df['Region'].value_counts()

Krasnodar Krai    47648
Stavropol Krai     3246
Rostov Oblast      2388
Moscow             2116
Crimea             1363
                  ...  
Bangui                1
Crete                 1
Flanders              1
Kvemo Kartli          1
Jiangxi               1
Name: Region, Length: 272, dtype: int64

In [4]:
df['Region'].value_counts().max()

47648

In [5]:
# Какой средний процент отказов (Bounce)
round(df['Bounce Rate'].mean(), 5)

0.00229

In [6]:
# С каких устройств чаще заходят на сайты
df['Device Category'].value_counts()

mobile     39465
desktop    24630
tablet      2067
Name: Device Category, dtype: int64

In [7]:
# Какие источники наиболее конвертируемые
df_pivot = df.pivot_table(
    index ='Source',
    values = 'Конверсия',
    aggfunc = 'sum').sort_values(by='Конверсия', ascending=False)
df_pivot.head()

,Конверсия
Source,
google,9106
yandex.ru,5668
yandex,5035
(direct),4790
BMW-keyauto-krd.ru,4151


In [8]:
# Рассчитайте ROMI (при расчете придумайте методологию расчета средней стоимость проданного автомобиля)
catalog = pd.read_excel('Аналитика_1_СемашкоМА.xlsx', sheet_name = 'Справочник')
catalog.head()

,Марка,Модель,Цена,Валюта,Маржинальность
0,Mercedes,gls,1.135135e+05,Доллар США,0.33
1,Mercedes,GT_AMG,1.743243e+05,Доллар США,0.34
2,BMW,x5,7.222222e+04,Евро,0.27
3,BMW,i3,5.166667e+04,Евро,0.35
4,Mercedes,с180,2.300000e+06,Рубль,0.20


In [9]:
catalog['Курс'] = 1

In [10]:
catalog.loc[catalog['Валюта'] == 'Доллар США', 'Курс'] = 93.34

In [11]:
catalog.loc[catalog['Валюта'] == 'Евро', 'Курс'] = 99.58

In [12]:
catalog['Цена в рублях'] = catalog['Курс']*catalog['Цена']

In [13]:
median_price = catalog['Цена в рублях'].median()
median_price

4300000.0

Использую в данном случае медианную стоимость для удобства расчета, т.к. она близка к средней стоимости в зависимости от числа проданных автомобилей (этот расчет см. в файле excel).
Среднее значение стоимости использовать некорректно, т.к. она сильно смещена в большую сторону.

In [14]:
crm = pd.read_excel('Аналитика_1_СемашкоМА.xlsx', sheet_name = 'Данные из CRM')
crm

,Client ID,Город,Просчет стоимости модели,Приход к диллеру,Продажа
0,GA1.2.1108606219.1572272784,Krasnodar,NaN,0.0,NaN
1,GA1.2.125017234.1571028033,Krasnodar,0.0,1.0,NaN
2,GA1.2.1823974094.1570697747,Krasnodar,NaN,NaN,NaN
3,GA1.2.1823974094.1570697747,Krasnodar,0.0,1.0,NaN
4,GA1.2.554197778.1572516039,Krasnodar,NaN,0.0,NaN
...,...,...,...,...,...
66157,GA1.2.809073096.1580964660,Voronezh,1.0,1.0,1.0
66158,GA1.2.914030393.1581675332,Saint Petersburg,NaN,NaN,NaN
66159,GA1.2.930162724.1578985942,Voronezh,NaN,NaN,NaN
66160,GA1.2.945678631.1572467756,Vladikavkaz,NaN,NaN,NaN


In [15]:
sales = crm['Продажа'].sum()
sales

4086.0

In [16]:
marketing_costs = df['Goal Value'].sum()
marketing_costs

868143913

In [17]:
romi = (sales*median_price - marketing_costs)/marketing_costs * 100
romi

1923.8349583405995

In [18]:
# Посчитайте выручку в рублях только по долларовым позициям
crm = crm[crm['Продажа']==1]
crm

,Client ID,Город,Просчет стоимости модели,Приход к диллеру,Продажа
12,GA1.2.1537731947.1571378524,Krasnodar,1.0,1.0,1.0
15,GA1.2.1611513157.1574872756,Krasnodar,1.0,1.0,1.0
28,GA1.2.259369071.1577459680,Krasnodar,1.0,1.0,1.0
38,GA1.2.640718727.1580984196,Krasnodar,1.0,1.0,1.0
39,GA1.2.933056671.1581701964,Krasnodar,1.0,1.0,1.0
...,...,...,...,...,...
66122,GA1.2.1249097580.1578555845,Stavropol,1.0,1.0,1.0
66133,GA1.2.1900101703.1578692959,Voronezh,1.0,1.0,1.0
66148,GA1.2.722639312.1578891070,Simferopol,1.0,1.0,1.0
66156,GA1.2.805131018.1579698795,Rostov-on-Don,1.0,1.0,1.0


In [19]:
crm = crm.merge(df.drop_duplicates(subset=['Client ID']), on = 'Client ID')
crm

,Client ID,Город,Просчет стоимости модели,Приход к диллеру,Продажа,Domain,City,Region,Browser,Device Category,...,Bounce Rate,Sessions,Goal Completion Location,Goal Completions,Goal Conversion Rate,Goal Value,Конверсия,Source conv.,Марка,Модель
0,GA1.2.1537731947.1571378524,Krasnodar,1.0,1.0,1.0,BMW-keyauto-krd.ru,Krasnodar,Krasnodar Krai,Samsung Internet,mobile,...,0.0,1,/promo/x1-rock/,1,1.0,40088,1,google_go_search,BMW,x1
1,GA1.2.1611513157.1574872756,Krasnodar,1.0,1.0,1.0,BMW-keyauto-krd.ru,Krasnodar,Krasnodar Krai,Chrome,mobile,...,0.0,1,/promo/x1-rock/,1,1.0,23226,1,google_go_search,BMW,x1
2,GA1.2.1611513157.1574872756,Krasnodar,1.0,1.0,1.0,BMW-keyauto-krd.ru,Krasnodar,Krasnodar Krai,Chrome,mobile,...,0.0,1,/promo/x1-rock/,1,1.0,23226,1,google_go_search,BMW,x1
3,GA1.2.259369071.1577459680,Krasnodar,1.0,1.0,1.0,BMW-keyauto-krd.ru,Krasnodar,Krasnodar Krai,Samsung Internet,mobile,...,0.0,2,/offers/buy/suv-winter2020/,1,0.5,17893,1,google_go_search,BMW,NaN
4,GA1.2.259369071.1577459680,Krasnodar,1.0,1.0,1.0,BMW-keyauto-krd.ru,Krasnodar,Krasnodar Krai,Samsung Internet,mobile,...,0.0,2,/offers/buy/suv-winter2020/,1,0.5,17893,1,google_go_search,BMW,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4081,GA1.2.960784881.1581156089,Stavropol,1.0,1.0,1.0,Mersedes-krasnodar.ru,Stavropol,Stavropol Krai,Safari,mobile,...,NaN,1,/special/purchase/?calltouch_tm=yd_c:36451549_...,1,1.0,26,1,YANDEX,Mercedes,NaN
4082,GA1.2.1900101703.1578692959,Voronezh,1.0,1.0,1.0,Mersedes-krasnodar.ru,Voronezh,Voronezh Oblast,Chrome,mobile,...,0.0,2,/models/cla/options/,1,0.5,11497,1,YANDEX,Mercedes,cla
4083,GA1.2.722639312.1578891070,Simferopol,1.0,1.0,1.0,Mersedes-krasnodar.ru,Simferopol,Crimea,Chrome,mobile,...,NaN,1,/special/purchase/?calltouch_tm=yd_c:36451549_...,1,1.0,9,1,YANDEX,Mercedes,NaN
4084,GA1.2.805131018.1579698795,Rostov-on-Don,1.0,1.0,1.0,Mersedes-krasnodar.ru,Voronezh,Voronezh Oblast,Chrome,desktop,...,0.0,1,(entrance),1,1.0,37406,1,yandex.ru,Mercedes,NaN


In [20]:
crm = crm.merge(catalog, on = 'Модель')
crm

,Client ID,Город,Просчет стоимости модели,Приход к диллеру,Продажа,Domain,City,Region,Browser,Device Category,...,Конверсия,Source conv.,Марка_x,Модель,Марка_y,Цена,Валюта,Маржинальность,Курс,Цена в рублях
0,GA1.2.1537731947.1571378524,Krasnodar,1.0,1.0,1.0,BMW-keyauto-krd.ru,Krasnodar,Krasnodar Krai,Samsung Internet,mobile,...,1,google_go_search,BMW,x1,BMW,2200000.0,Рубль,0.22,1.0,2200000.0
1,GA1.2.1611513157.1574872756,Krasnodar,1.0,1.0,1.0,BMW-keyauto-krd.ru,Krasnodar,Krasnodar Krai,Chrome,mobile,...,1,google_go_search,BMW,x1,BMW,2200000.0,Рубль,0.22,1.0,2200000.0
2,GA1.2.1611513157.1574872756,Krasnodar,1.0,1.0,1.0,BMW-keyauto-krd.ru,Krasnodar,Krasnodar Krai,Chrome,mobile,...,1,google_go_search,BMW,x1,BMW,2200000.0,Рубль,0.22,1.0,2200000.0
3,GA1.2.604399816.1575047454,Krasnodar,1.0,1.0,1.0,BMW-keyauto-krd.ru,Krasnodar,Krasnodar Krai,Android Webview,mobile,...,1,carsdo.ru,BMW,x1,BMW,2200000.0,Рубль,0.22,1.0,2200000.0
4,GA1.2.89633625.1579455564,Krasnodar,1.0,1.0,1.0,BMW-keyauto-krd.ru,Krasnodar,Krasnodar Krai,Samsung Internet,mobile,...,1,carsdo.ru,BMW,x1,BMW,2200000.0,Рубль,0.22,1.0,2200000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,GA1.2.1137856979.1579711782,Antalya,1.0,1.0,1.0,Mersedes-krasnodar.ru,Antalya,Antalya,Firefox,desktop,...,1,carsdo.ru,Mercedes,GLE,Mercedes,7200000.0,Рубль,0.29,1.0,7200000.0
611,GA1.2.414569633.1577655391,Saint Petersburg,1.0,1.0,1.0,BMW-keyauto-krd.ru,Saint Petersburg,Saint Petersburg,Chrome,tablet,...,1,google_go_search,BMW,GLE,Mercedes,7200000.0,Рубль,0.29,1.0,7200000.0
612,GA1.2.758532568.1578958024,Krasnodar,1.0,1.0,1.0,Mersedes-krasnodar.ru,Krasnodar,Krasnodar Krai,Samsung Internet,tablet,...,1,YANDEX,Mercedes,GLE,Mercedes,7200000.0,Рубль,0.29,1.0,7200000.0
613,GA1.2.297521342.1581517106,Krasnodar,1.0,1.0,1.0,Mersedes-krasnodar.ru,Krasnodar,Krasnodar Krai,Chrome,desktop,...,0,YANDEX,Mercedes,GLE,Mercedes,7200000.0,Рубль,0.29,1.0,7200000.0


In [21]:
revenue  = round(crm['Цена в рублях'][crm['Валюта'] == 'Доллар США'].sum(), 2)
revenue

656911783.78

In [22]:
# Определите, какой источник трафика наиболее выгоден для компании по текущим данным
crm_pivot = crm.pivot_table(
    index ='Source',
    values = 'Продажа',
    aggfunc = 'sum').sort_values(by='Продажа', ascending=False)
crm_pivot.head()

,Продажа
Source,
yandex,164.0
google,118.0
(direct),92.0
yandex.ru,91.0
IG,69.0


**Ответьте на вопрос: каких показателей не хватает, чтобы посчитать чистую прибыль?**

чистая прибыль = выручка – COGC(себестоимость, ее можно высчитать из имеющейся маржинальности) – Marketing costs(расходы на маркетинг, они есть) - Ops costs(Операционные расходы, их не хватает)

Среди операционных расходов есть расходы на ФОТ(его мы не знаем), налоги (их можно посчитать согласно законодательству), аренда, амортизация, лицензии и прочие админимтративные расходы (их мы тоже не знаем) 